In [13]:
pip install pydotplus


  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24572 sha256=cd0b8f5474fa644176a093dc7ff2ea24d273b287c29f13708381f44415f97ca9
  Stored in directory: c:\users\mahe\appdata\local\pip\cache\wheels\1f\5c\ba\f931f74fcac8f48b18ae597279203b1c1f92fc76249c2b6f66
Successfully built pydotplus


In [10]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report 
from imblearn.over_sampling import SMOTE
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('final.csv')
df=df.drop(columns=['Unnamed: 0'])
df.head()

,date,lat,lon,mask,lst,lstn,ndvi,vcr,prec,ma_cat
0,2019-06-02,39.737500,-122.295833,9.0,310.26000,290.921693,0.3108,0.59,0.718065,1
1,2019-06-10,39.737500,-122.295833,5.0,323.92000,290.921693,0.2526,0.59,0.000000,0
2,2019-06-18,39.737500,-122.295833,5.0,309.33877,297.900000,0.2526,0.59,0.000000,0
3,2019-06-02,39.729167,-122.295833,9.0,310.36000,290.921693,0.3198,0.57,0.718065,1
4,2019-06-10,39.729167,-122.295833,5.0,321.90000,290.921693,0.2670,0.57,0.000000,0


In [21]:
X = df[['vcr', 'lst','ndvi','lstn','prec']] .values  #.astype(float)
x=df[['vcr', 'lst','ndvi','lstn','prec']]
X[0:5]

array([[5.90000000e-01, 3.10260000e+02, 3.10800000e-01, 2.90921693e+02,
        7.18065143e-01],
       [5.90000000e-01, 3.23920000e+02, 2.52600000e-01, 2.90921693e+02,
        0.00000000e+00],
       [5.90000000e-01, 3.09338770e+02, 2.52600000e-01, 2.97900000e+02,
        0.00000000e+00],
       [5.70000000e-01, 3.10360000e+02, 3.19800000e-01, 2.90921693e+02,
        7.18065143e-01],
       [5.70000000e-01, 3.21900000e+02, 2.67000000e-01, 2.90921693e+02,
        0.00000000e+00]])

In [22]:
y = df['ma_cat'].values
y[0:5]

array([1, 0, 0, 1, 0], dtype=int64)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

# Before SMOTE

In [24]:
print(sum(y_train == 1))
print(sum(y_train == 0))

78
1255


In [25]:
dtr = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
dtr # it shows the default parameters

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [26]:
dtr.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [27]:
predTree = dtr.predict(X_test)

In [28]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

DecisionTrees's Accuracy:  0.9423076923076923


In [29]:
df.columns[4:7]

Index(['lst', 'lstn', 'ndvi'], dtype='object')

In [30]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

ModuleNotFoundError: No module named 'sklearn.externals.six'

In [18]:
dot_data = StringIO()
filename = "firetree.png"
featureNames = df.columns[4:7]
targetNames = df["ma_cat"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

TypeError: must be str, not numpy.int64

# After SMOTE

In [31]:
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)

In [32]:
print(sum(y_train == 1))
print(sum(y_train == 0))

1255
1255


In [33]:
dtr.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [34]:
y_pred=dtr.predict(X_test)

In [35]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, y_pred))

DecisionTrees's Accuracy:  0.6748251748251748


In [36]:
feature_imp = pd.Series(dtr.feature_importances_,index=x.columns.values).sort_values(ascending=False)
feature_imp

lst     0.370483
prec    0.366599
lstn    0.153635
ndvi    0.085775
vcr     0.023508
dtype: float64

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.68      0.80       541
           1       0.10      0.61      0.17        31

    accuracy                           0.67       572
   macro avg       0.53      0.65      0.48       572
weighted avg       0.92      0.67      0.76       572

